In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
#pip install langchain

In [4]:
#pip install openai

In [6]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

In [12]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("OPEN_API_KEY")

In [93]:
llm=ChatOpenAI(openai_api_key=secret_value_0,model_name="gpt-3.5-turbo-0125",temperature=0)

In [94]:
solutions_template="""
Generate {num_solutions} distinct solutions for {problem}. Consider factors like {factors}.

Solutions:
"""
solutions_prompt=PromptTemplate(
    template=solutions_template,
    input_variables=["problem","factors","num_solutions"]
)

In [95]:
evaluation_template="""
Evaluate each solution in {Solutions} by analyzing pros,cons,
feasibility, and probability of success.

Evaluations:
"""
evaluation_prompt=PromptTemplate(
    template=evaluation_template,
    input_variables=["Solutions"]
)

In [96]:
reasoning_template="""
For the most promising solutions in {evaluations}, explain scenarios, 
implementation strategies, partnerships needed, and handling potential 
obstacles.

Enhanced Reasoning:
"""
reasoning_prompt=PromptTemplate(
    template=reasoning_template,
    input_variables=["evaluations"]
)


In [97]:
ranking_template="""
Based on the evaluations and reasoning, rank the solutions in {enhanced_reasoning} from most to least promising.

Ranked Solutions:
"""
ranking_prompt=PromptTemplate(
    template=ranking_template,
    input_variables=["enhanced_reasoning"]
)

In [98]:
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI

solutions_chain=LLMChain(
    llm=llm,
    prompt=solutions_prompt,
    output_key="Solutions"
)
evaluation_chain=LLMChain(
    llm=llm,
    prompt=evaluation_prompt,
    output_key="evaluations"
)
reasoning_chain=LLMChain(
    llm=llm,
    prompt=reasoning_prompt,
    output_key="enhanced_reasoning"
)
ranking_chain=LLMChain(
    llm=llm,
    prompt=ranking_prompt,
    output_key="ranked_solutions"
)



In [99]:
from langchain.chains import SequentialChain
tot_chain=SequentialChain(
    chains=[solutions_chain,evaluation_chain,reasoning_chain,ranking_chain],
    input_variables=["problem","factors","num_solutions"],
    output_variables=["ranked_solutions"]
)


In [104]:
print(tot_chain.run(
 problem="Retrieval-Augmented Generation",
 factors="Requirements for high task performance, low token use, and few calls to the LLM",
 num_solutions=3
))

1. Scenario 3: Developing a hybrid model
2. Scenario 1: Implementing a retrieval system
3. Scenario 2: Using reinforcement learning
